In [45]:
using DrWatson
@quickactivate "dare"

using ReinforcementLearning
using IntervalSets
using LinearAlgebra
using ControlSystems
using CUDA
using DataStructures
using DataFrames
using PlotlyJS

include(srcdir("nodeconstructor.jl"))
include(srcdir("data_hook.jl"))
include(srcdir("env.jl"))
include(srcdir("sin_policy.jl"))
include(srcdir("plotting.jl"))
include(srcdir("custom_control.jl"));

### Define CM Matrix

In [46]:
CM = [0.0   1.0    0.0    0.0    2.0    3.0    4.0    5.0   6.0   7.0
     -1.0   0.0    0.0    0.0    0.0    8.0    0.0    0.0   0.0   9.0
      0.0   0.0    0.0    0.0   10.0   11.0   12.0   13.0   0.0  14.0
      0.0   0.0    0.0    0.0   15.0   16.0   17.0    0.0  18.0  19.0
     -2.0   0.0  -10.0  -15.0    0.0   20.0    0.0   21.0  22.0   0.0
     -3.0  -8.0  -11.0  -16.0  -20.0    0.0   23.0   24.0  25.0  26.0
     -4.0   0.0  -12.0  -17.0    0.0  -23.0    0.0   27.0  28.0  29.0
     -5.0   0.0  -13.0    0.0  -21.0  -24.0  -27.0    0.0   0.0  30.0
     -6.0   0.0    0.0  -18.0  -22.0  -25.0  -28.0    0.0   0.0   0.0
     -7.0  -9.0  -14.0  -19.0    0.0  -26.0  -29.0  -30.0   0.0   0.0]



10×10 Matrix{Float64}:
  0.0   1.0    0.0    0.0    2.0    3.0    4.0    5.0   6.0   7.0
 -1.0   0.0    0.0    0.0    0.0    8.0    0.0    0.0   0.0   9.0
  0.0   0.0    0.0    0.0   10.0   11.0   12.0   13.0   0.0  14.0
  0.0   0.0    0.0    0.0   15.0   16.0   17.0    0.0  18.0  19.0
 -2.0   0.0  -10.0  -15.0    0.0   20.0    0.0   21.0  22.0   0.0
 -3.0  -8.0  -11.0  -16.0  -20.0    0.0   23.0   24.0  25.0  26.0
 -4.0   0.0  -12.0  -17.0    0.0  -23.0    0.0   27.0  28.0  29.0
 -5.0   0.0  -13.0    0.0  -21.0  -24.0  -27.0    0.0   0.0  30.0
 -6.0   0.0    0.0  -18.0  -22.0  -25.0  -28.0    0.0   0.0   0.0
 -7.0  -9.0  -14.0  -19.0    0.0  -26.0  -29.0  -30.0   0.0   0.0

### Create the environment and thus the Nodeconstructor with the CM Matrix

In [47]:
env = SimEnv(num_sources=5, num_loads=5,CM=CM)

INFO: v_dc = 350V will get applied to all actions
INFO: Generating standard norm_array from nodeconstructor


# SimEnv

## Traits

| Trait Type        |                  Value |
|:----------------- | ----------------------:|
| NumAgentStyle     |          SingleAgent() |
| DynamicStyle      |           Sequential() |
| InformationStyle  | ImperfectInformation() |
| ChanceStyle       |           Stochastic() |
| RewardStyle       |           StepReward() |
| UtilityStyle      |           GeneralSum() |
| ActionStyle       |     MinimalActionSet() |
| StateStyle        |     Observation{Any}() |
| DefaultStateStyle |     Observation{Any}() |

## Is Environment Terminated?

No

## State Space

`Space{Vector{ClosedInterval{Float64}}}(ClosedInterval{Float64}[-1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0])`

## Action Space

`Space{Vector{ClosedInterval{Float64}}}(ClosedInterval{Float64}[-1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0, -1.0..1.0])`

## Current State

```
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
```


### Define Policy

In [48]:
policy = RandomPolicy(action_space(env))
#policy = sin_policy()

typename(RandomPolicy)
├─ action_space => typename(Space)
│  └─ s => 15-element Vector{ClosedInterval{Float64}}
└─ rng => typename(Random._GLOBAL_RNG)


### Create Hook:
#### 2 Options:
   - pass the states to be stored to "collect_state_ids"
       - need to know the exact Name of the states in the Nodeconstructor
       - only states at L and C 
   - hand in the ID(Number) of Sources, Loads and Cables which states shall be stored
       - computes Voltage and Current at all R, L and C

In [49]:
env.action_ids

15-element Vector{String}:
 "source1_u_a"
 "source2_u_a"
 "source3_u_a"
 "source4_u_a"
 "source5_u_a"
 "source1_u_b"
 "source2_u_b"
 "source3_u_b"
 "source4_u_b"
 "source5_u_b"
 "source1_u_c"
 "source2_u_c"
 "source3_u_c"
 "source4_u_c"
 "source5_u_c"

In [50]:
hook = DataHook(collect_state_ids=[], collect_sources=[1,2], collect_cables=[1,2,3],collect_loads=[1], collect_state_paras=get_state_paras(env.nc))

DataHook(false, "episode_data/", nothing, nothing, Any[0.0012582110669914284, 2.025090858127555e-5, 0.0012582110669914284, 9.260887466775631e-6, 0.0007175106173461386, 3.384504881313394e-5, 0.0007175106173461386, 4.0311700685012376e-6, 0.0012066027454164375, 2.0156042763641025e-5  …  0.002239297662963504, 88.06301224566828, 2.571, 36.59100890662512, 22.706, 23.598008623470523, 61.664, 85.11100566733154, 53.852, 86.7130099408717], Any[], Any[], Any[], [1, 2], [1, 2, 3], [1], Any[], Any[], Any[], 0×0 DataFrame, 0×0 DataFrame, 1, false, Vector{Float64}[], [0.0], String[], false, nothing, -1.0e6, 0, nothing, false)

In [51]:
hook.collect_action_ids

Any[]

In [52]:
run(policy, env, StopAfterEpisode(1), hook)

Any["source1_i_R1_a", "source1_i_R_C_a", "source1_i_R2_a", "source1_i_R_C_cables_a", "source1_i_R1_b", "source1_i_R_C_b", "source1_i_R2_b", "source1_i_R_C_cables_b", "source1_i_R1_c", "source1_i_R_C_c", "source1_i_R2_c", "source1_i_R_C_cables_c", "source2_i_R1_a", "source2_i_R_C_a", "source2_i_R2_a", "source2_i_R_C_cables_a", "source2_i_R1_b", "source2_i_R_C_b", "source2_i_R2_b", "source2_i_R_C_cables_b", "source2_i_R1_c", "source2_i_R_C_c", "source2_i_R2_c", "source2_i_R_C_cables_c", "cable1_i_R_a", "cable1_i_R_b", "cable1_i_R_c", "cable2_i_R_a", "cable2_i_R_b", "cable2_i_R_c", "cable3_i_R_a", "cable3_i_R_b", "cable3_i_R_c", ("load1_i_R_a", "load1_i_C_a", "load1_i_C_cables_a"), ("load1_i_R_b", "load1_i_C_b", "load1_i_C_cables_b"), ("load1_i_R_c", "load1_i_C_c", "load1_i_C_cables_c")]

DataHook(false, "episode_data/", [-400.45066003244716 -794.7792117193422 … 0.0 0.0; 49380.5004346631 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [794.7792117193422 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Any[0.0012582110669914284, 2.025090858127555e-5, 0.0012582110669914284, 9.260887466775631e-6, 0.0007175106173461386, 3.384504881313394e-5, 0.0007175106173461386, 4.0311700685012376e-6, 0.0012066027454164375, 2.0156042763641025e-5  …  0.002239297662963504, 88.06301224566828, 2.571, 36.59100890662512, 22.706, 23.598008623470523, 61.664, 85.11100566733154, 53.852, 86.7130099408717], Any[0.5032844267965714, 0.0005670254402757154, 0.5032844267965714, 0.0005670254402757154, 0.5032844267965714, 0.0005670254402757154, 0.5032844267965714, 0.0005670254402757154, 0.5032844267965714, 0.0005670254402757154  …  6.845868526153175, 5.315827887152063, 5.315827887152063, 5.315827887152063, 6.59828375274393, 6.59828375274393, 6.59828375274393, (67214.038

## Labeling in Dataframe:

In [26]:
env.nc.parameters["source"][1]

Dict{Any, Any} with 11 entries:
  "L1"    => 0.00571151
  "C"     => 4.17767e-6
  "fltr"  => "LCL"
  "pwr"   => 5000.0
  "L2"    => 0.00571151
  "R_C"   => 0.000116975
  "v_rip" => 0.0140926
  "vdc"   => 700
  "R1"    => 2.2846
  "i_rip" => 0.149493
  "R2"    => 2.2846

In [27]:
env.nc.parameters["cable"][1]

Dict{Any, Any} with 3 entries:
  "C" => 9.85516e-7
  "L" => 0.000650441
  "R" => 1.77886

In [28]:
env.nc.parameters["load"][1]

Dict{Any, Any} with 4 entries:
  "C"         => 43.171
  "L"         => 63.663
  "R"         => 11307.7
  "impedance" => "RLC"

## Column Title in Dataframe:
### Sources: source(id) _ (i/u) _ (Component-name)_(Phase)
### Cables: cable(id) _ (i/u) _ (Component-Name)_(Phase)
### Loads: load(id) _ (i/u) _ (Component-Name)_(Phase)


### Examples:

In [57]:
state_name1= "source2_i_R1_a"
state_name2= "cable3_u_R_b"
state_name3= "load1_i_C_a"

hook.df[:, :source2_u_a]

500-element Vector{Float64}:
    0.0
 -112.41458697211426
  -97.20009151843372
 -172.92964456431176
  165.2073928172253
   -9.15148184858115
 -149.8444411206208
 -129.64812215521002
   14.128055762210268
   84.33511238498669
    ⋮
   43.664757437201246
  -10.99480573320385
 -154.83503308570076
  -87.92965374393502
  138.4183148607575
  134.88834598218386
 -167.40896475600616
  142.56283345938155
   93.81710821708327
  -56.57645037323928